In [62]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import sqlite3
import re
import rapidfuzz

In [148]:
# Get crinacle's ranking table
r = requests.get('https://crinacle.com/rankings/iems/')
soup = BeautifulSoup(r.text, 'html.parser')
# Find the table headers
header_row = soup.find('tr')
headers = [th.text for th in header_row.find_all('th')]
rows = soup.find_all('tr') # Find all the table rows
data=[]
for row in rows: # Iterate through rows
    tds = row.find_all('td') # Find all table data (td) elements within each table row 'tr'
    row_data = [td.text for td in tds] # Extract row data from each td
    data.append(row_data)
# Create a pandas DataFrame for crinacle's ranking list.
iefdf = pd.DataFrame(data, columns=headers)
iefdf

,Rank,Value Rating,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Status,Ranksort,Tonesort,Techsort,Pricesort
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,S-,★,Elysian Annihilator (2021),3700,U-shaped,,S-,S,2EST 4BA 1DD,Zeppelin & Co demo unit,10,7,3,3700
2,S-,★★,"Hidition Viento(B-mode, custom)",950,Variable,Best-in-class tuning and tonal balance with ne...,S+,A+,4BA,Zeppelin & Co demo unit,14,1,13,950
3,S-,★★,ThieAudio Monarch Mk2,1000,Neutral with bass boost,,S+,A+,2EST 6BA 1DD,Available for demo at The Hangout,14,1,13,1000
4,A+,★,Hidition NT6,1050,Neutral,"If Etymotic made a multi-BA IEM, this would be...",S-,A+,6BA,Null Audio demo unit,20,7,13,1050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,F,,VE Monk IE Biggie,Discont.,Dark,Absolute mud and congestion. Could work as a s...,F,E,DD,Third party unit,1266,844,422,99999
2140,F,,Oppo MH130,14,Complete failure,The horribleness of this can only be experienc...,F,F,DD,Third party unit,1688,844,844,14
2141,F,,Delta Air complimentary earphones,0,Complete failure,It's a telephone in IEM form. It is physically...,F,F,DD,Third party unit,1688,844,844,0
2142,F,,SIA complimentary earphones,0,Dark,Its full potential can only be unleashed by an...,F,F,DD,Personal unit,1688,844,844,0


In [149]:
# Remove the space in the ' Rank' column
iefdf=iefdf.rename(columns={' Rank':'Normalized Grade', 'Price (MSRP)':'Price'})
# Formatting begins heres
iefdf = iefdf.iloc[1:]
iefdf = iefdf.reset_index()
iefdf = iefdf.drop(iefdf.index[-1]) #drops last row

In [150]:
iefdf=iefdf.rename(columns={'Technical Grade':'Tech Grade'})
iefdf['Normalized Float'] = iefdf['Normalized Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0, 'F':0})
iefdf['Tone Float']=iefdf['Tone Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0, 'F':0})

iefdf['Tech Float']=iefdf['Tech Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0, 'F':0})

iefdf['Normalized Float and Grade'] = iefdf['Normalized Float'].astype(str) + ' (' + iefdf['Normalized Grade'].astype(str) + ')'
iefdf['Tone Float and Grade'] = iefdf['Tone Float'].astype(str) + ' (' + iefdf['Tone Grade'].astype(str) + ')'
iefdf['Tech Float and Grade'] = iefdf['Tech Float'].astype(str) + ' (' + iefdf['Tech Grade'].astype(str) + ')'

# remove the index column from iefdf as it is redundant, and remove the 'Value Rating' as we will be calculating it ourselves later.
# Also, remove Pricesort, Ranksort, Tonesort, Techsort, 'Based on', columns.
iefdf = iefdf.drop(columns=['index', 'Value Rating', 'Pricesort', 'Ranksort', 'Tonesort', 'Techsort'])
# add a column "list" that will be used to identify the list the IEM is from. This will be used later to identify the IEMs that are in both lists.
# Use iefdf['list'] = 'ief' to add the list column to the iefdf DataFrame.
iefdf['List'] = 'ief'

In [151]:
iefdf

,Normalized Grade,Model,Price,Signature,Comments,Tone Grade,Tech Grade,Setup,Status,Normalized Float,Tone Float,Tech Float,Normalized Float and Grade,Tone Float and Grade,Tech Float and Grade,List
0,S-,Elysian Annihilator (2021),3700,U-shaped,,S-,S,2EST 4BA 1DD,Zeppelin & Co demo unit,7.9,7.9,8.7,7.9 (S-),7.9 (S-),8.7 (S),ief
1,S-,"Hidition Viento(B-mode, custom)",950,Variable,Best-in-class tuning and tonal balance with ne...,S+,A+,4BA,Zeppelin & Co demo unit,7.9,9.0,7,7.9 (S-),9.0 (S+),7 (A+),ief
2,S-,ThieAudio Monarch Mk2,1000,Neutral with bass boost,,S+,A+,2EST 6BA 1DD,Available for demo at The Hangout,7.9,9.0,7,7.9 (S-),9.0 (S+),7 (A+),ief
3,A+,Hidition NT6,1050,Neutral,"If Etymotic made a multi-BA IEM, this would be...",S-,A+,6BA,Null Audio demo unit,7,7.9,7,7 (A+),7.9 (S-),7 (A+),ief
4,A+,ThieAudio V16 Divinity,1500,Neutral with bass boost,,S-,A+,16BA,Available for demo at The Hangout,7,7.9,7,7 (A+),7.9 (S-),7 (A+),ief
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,E,VE Monk IE Smalls,Discont.,Dark,Sounds exactly like what would happen if you s...,E,E,DD,Third party unit,0.5,0.5,0.5,0.5 (E),0.5 (E),0.5 (E),ief
2138,F,VE Monk IE Biggie,Discont.,Dark,Absolute mud and congestion. Could work as a s...,F,E,DD,Third party unit,0,0,0.5,0 (F),0 (F),0.5 (E),ief
2139,F,Oppo MH130,14,Complete failure,The horribleness of this can only be experienc...,F,F,DD,Third party unit,0,0,0,0 (F),0 (F),0 (F),ief
2140,F,Delta Air complimentary earphones,0,Complete failure,It's a telephone in IEM form. It is physically...,F,F,DD,Third party unit,0,0,0,0 (F),0 (F),0 (F),ief


In [152]:
# Make a request to the website
r = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vTEdqzrEci3pGaAOu09zmYOhBtlrEPRjds3jXVgOuaN7vWQ7JWM1FNQEBeHqiPq7A/pubhtml")
r_text = r.text
# Create a BeautifulSoup object and specify html parser
soup = BeautifulSoup(r_text, 'html.parser')
tables = soup.find_all('table')
table = tables[0]
antdf = pd.read_html(str(table), header=0)[0]

In [153]:
antdf = antdf.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 10', 'Unnamed: 11'])
# Make the second row the header
antdf.columns = antdf.iloc[1]
antdf=antdf.iloc[2:] # drop the first two rows

In [154]:
# Create new column called Normalized Grade Float that takes the value in the Normalized Grade column and turns it into a float, where S+=9, S=8.7, S-=7.9, A+=7, A=6.5, A-=6, B+=5.5, B=5, B-=4.5, C+=4, C=3.5, C-=3, D+=2.5, D=2, D-=1.5, E+=1, E=0.5, E-=0.2, F+=0.1, F=0
antdf['Normalized Float'] = antdf['Normalized Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0.2, 'F+':0.1, 'F':0})
# Rename technical score to tech grade and Tonality Score to Tone Grade
antdf = antdf.rename(columns={'Technical Score':'Tech Grade', 'Tonality Score':'Tone Grade', 'Preference Score':'Preference Grade', 'IEM':'Model', 'Price (USD)':'iefdf'})
antdf['Tone Float'] = antdf['Tone Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0.2, 'F+':0.1, 'F':0})
antdf['Tech Float'] = antdf['Tech Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0.2, 'F+':0.1, 'F':0})
antdf['Preference Float'] = antdf['Preference Grade'].replace({'S+':9.0, 'S':8.7, 'S-':7.9, 'A+':7, 'A':6.5, 'A-':6, 'B+':5.5, 'B':5, 'B-':4.5, 'C+':4, 'C':3.5, 'C-':3, 'D+':2.5, 'D':2, 'D-':1.5, 'E+':1, 'E':0.5, 'E-':0.2, 'F+':0.1, 'F':0})
# Create a column that combines combines the Normalized Grade combines the normalized grade and the normalized grade float, putting the float in brackets.
antdf['Normalized Float and Grade'] = antdf['Normalized Float'].astype(str) + ' (' + antdf['Normalized Grade'].astype(str) + ')'
antdf['Tone Float and Grade'] = antdf['Tone Float'].astype(str) + ' (' + antdf['Tone Grade'].astype(str) + ')'
antdf['Tech Float and Grade'] = antdf['Tech Float'].astype(str) + ' (' + antdf['Tech Grade'].astype(str) + ')'
antdf['Preference Float and Grade'] = antdf['Preference Float'].astype(str) + ' (' + antdf['Preference Grade'].astype(str) + ')'
# Add list column for antdf
antdf['List'] = 'ant'
antdf.head()

1,Model,iefdf,Normalized Grade,Tone Grade,Tech Grade,Preference Grade,Normalized Float,Tone Float,Tech Float,Preference Float,Normalized Float and Grade,Tone Float and Grade,Tech Float and Grade,Preference Float and Grade,List
2,Subtonic Storm,5200,S+,S+,S+,S+,9.0,9.0,9.0,9.0,9.0 (S+),9.0 (S+),9.0 (S+),9.0 (S+),ant
3,Empire Ears Odin,3400,S,S-,S+,S+,8.7,7.9,9.0,9.0,8.7 (S),7.9 (S-),9.0 (S+),9.0 (S+),ant
4,64 Audio U12T (M15),2000,S-,S-,S,S,7.9,7.9,8.7,8.7,7.9 (S-),7.9 (S-),8.7 (S),8.7 (S),ant
5,Thieaudio Monarch MK2,1000,S-,S,S-,S-,7.9,8.7,7.9,7.9,7.9 (S-),8.7 (S),7.9 (S-),7.9 (S-),ant
6,Hidition Viento-B CIEM,990,A+,S+,A,S-,7,9.0,6.5,7.9,7 (A+),9.0 (S+),6.5 (A),7.9 (S-),ant


In [155]:
cogdf = pd.read_csv('https://docs.google.com/spreadsheets/d/1pUCELfWO-G33u82H42J8G_WX1odnOYBJsBNbVskQVt8/export?format=csv')

In [156]:
# drop cols 1, 2, 3, 17, 18, 19, 20, 21, 23, 24
cogdf = cogdf.drop(columns=['Unnamed: 0', 'Rank', 'Unnamed: 3', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Based On'])
cogdf = cogdf.iloc[:cogdf[cogdf['IEM'].isnull()].index[0]] # Include no rows after the first row with an empty IEM column.

In [157]:
# Make the "Final Score" column float type
cogdf['Final Score'] = cogdf['Final Score'].astype(float)
cogdf['Tonality'] = cogdf['Tonality'].astype(float)
cogdf['Tech'] = cogdf['Tech'].astype(float)
cogdf['Bias '] = cogdf['Bias '].astype(float)

In [158]:
# Rename 'Final Score' to 'Normalized Float', Tonality to 'Tone Float', Tech to 'Tech Float', and 'Bias ' to 'Preference Float'
cogdf=cogdf.rename(columns={'IEM':'Model', 'Final Score':'Normalized Float', 'Tonality':'Tone Float', 'Tech':'Tech Float', 'Bias ':'Preference Float'})
# Add a column named Grade that takes the float in Final Score and assigns it to a grade depending on whether it's greater or equal to 8.7 for S, 7.9 for S-, 7 for A+, 6.5 for A, 6 for A-, 5.5 for B+, 5 for B, 4.5 for B-, 4 for C+, 3.5 for C, 3 for C-, 2.5 for D+, 2 for D, 1.5 for D-, 1 for E+, 0.5 for E, 0.2 for E-, 0.1 for F+, 0 for F
cogdf['Normalized Grade'] = cogdf['Normalized Float'].apply(lambda x: 'S' if x >= 8.7 else 'S-' if x >= 7.9 else 'A+' if x >= 7 else 'A' if x >= 6.5 else 'A-' if x >= 6 else 'B+' if x >= 5.5 else 'B' if x >= 5 else 'B-' if x >= 4.5 else 'C+' if x >= 4 else 'C' if x >= 3.5 else 'C-' if x >= 3 else 'D+' if x >= 2.5 else 'D' if x >= 2 else 'D-' if x >= 1.5 else 'E+' if x >= 1 else 'E' if x >= 0.5 else 'E-' if x >= 0.2 else 'F+' if x >= 0.1 else 'F')
# Do the same for the Tonality column, creating a column named Tone Grade
cogdf['Tone Grade'] = cogdf['Tone Float'].apply(lambda x: 'S' if x >= 8.7 else 'S-' if x >= 7.9 else 'A+' if x >= 7 else 'A' if x >= 6.5 else 'A-' if x >= 6 else 'B+' if x >= 5.5 else 'B' if x >= 5 else 'B-' if x >= 4.5 else 'C+' if x >= 4 else 'C' if x >= 3.5 else 'C-' if x >= 3 else 'D+' if x >= 2.5 else 'D' if x >= 2 else 'D-' if x >= 1.5 else 'E+' if x >= 1 else 'E' if x >= 0.5 else 'E-' if x >= 0.2 else 'F+' if x >= 0.1 else 'F')
# Do the same for the Tech column, creating a column named Tech Grade
cogdf['Tech Grade'] = cogdf['Tech Float'].apply(lambda x: 'S' if x >= 8.7 else 'S-' if x >= 7.9 else 'A+' if x >= 7 else 'A' if x >= 6.5 else 'A-' if x >= 6 else 'B+' if x >= 5.5 else 'B' if x >= 5 else 'B-' if x >= 4.5 else 'C+' if x >= 4 else 'C' if x >= 3.5 else 'C-' if x >= 3 else 'D+' if x >= 2.5 else 'D' if x >= 2 else 'D-' if x >= 1.5 else 'E+' if x >= 1 else 'E' if x >= 0.5 else 'E-' if x >= 0.2 else 'F+' if x >= 0.1 else 'F')
# rename the 'Bias ' column to 'Preference Float'
cogdf['Preference Grade'] = cogdf['Preference Float'].apply(lambda x: 'S' if x >= 8.7 else 'S-' if x >= 7.9 else 'A+' if x >= 7 else 'A' if x >= 6.5 else 'A-' if x >= 6 else 'B+' if x >= 5.5 else 'B' if x >= 5 else 'B-' if x >= 4.5 else 'C+' if x >= 4 else 'C' if x >= 3.5 else 'C-' if x >= 3 else 'D+' if x >= 2.5 else 'D' if x >= 2 else 'D-' if x >= 1.5 else 'E+' if x >= 1 else 'E' if x >= 0.5 else 'E-' if x >= 0.2 else 'F+' if x >= 0.1 else 'F')
# Create column combining the Final Score and Grade, putting the grade in brackets.
cogdf['Normalized Float and Grade'] = cogdf['Normalized Float'].astype(str) + ' (' + cogdf['Normalized Grade'].astype(str) + ')'
# Do the same for the Tonality column
cogdf['Tone Float and Grade'] = cogdf['Tone Float'].astype(str) + ' (' + cogdf['Tone Grade'].astype(str) + ')'
# Do the same for the Tech column
cogdf['Tech Float and Grade'] = cogdf['Tech Float'].astype(str) + ' (' + cogdf['Tech Grade'].astype(str) + ')'
# Do the same for the Bias column
cogdf['Preference Float and Grade'] = cogdf['Preference Float'].astype(str) + ' (' + cogdf['Preference Grade'].astype(str) + ')'
# Add list column for cogdf
cogdf['List'] = 'cog'

In [159]:
cogdf

,Model,Normalized Float,Tone Float,Bass,Midrange,Treble,Tech Float,Detail,Imaging,Dynamics,...,Comments,Normalized Grade,Tone Grade,Tech Grade,Preference Grade,Normalized Float and Grade,Tone Float and Grade,Tech Float and Grade,Preference Float and Grade,List
0,Subtonic Storm,9.0,8.67,9,8.0,9.0,9.33,10.0,8.0,10.0,...,Easily at the top for resolution and dynamic c...,S,S-,S,S,9.0 (S),8.67 (S-),9.33 (S),9.0 (S),cog
1,64 Audio U12t ⭑,8.7,8.00,8,9.0,7.0,9.00,9.0,9.0,9.0,...,Terrific dynamic range and soundstage depth; t...,S,S-,S,S,8.7 (S),8.0 (S-),9.0 (S),9.0 (S),cog
2,Elysian Annihilator 2021,8.7,8.33,7,8.0,10.0,8.67,10.0,7.0,9.0,...,Agressive listen with excellent resolution and...,S,S-,S-,S,8.7 (S),8.33 (S-),8.67 (S-),9.0 (S),cog
3,64 Audio Tia Trio ⭑,8.1,8.00,10,7.0,7.0,8.33,8.0,8.0,9.0,...,Rich bass and out-of-head imaging.,S-,S-,S-,S-,8.1 (S-),8.0 (S-),8.33 (S-),8.0 (S-),cog
4,Empire Ears Odin,8.0,8.33,9,8.0,8.0,7.67,10.0,7.0,6.0,...,High-clarity signature that invokes a sense of...,S-,S-,A+,S-,8.0 (S-),8.33 (S-),7.67 (A+),8.0 (S-),cog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,Shozy Black Hole Mini,2.0,2.00,2,2.0,2.0,2.00,2.0,2.0,2.0,...,"Midrange, especially the upper-midrange, is li...",D,D,D,D,2.0 (D),2.0 (D),2.0 (D),2.0 (D),cog
306,Nothing Ear 2,2.0,1.00,NaN,NaN,NaN,3.00,NaN,NaN,NaN,...,Absolute cacophany of sound; comparable to blo...,D,E+,C-,D,2.0 (D),1.0 (E+),3.0 (C-),2.0 (D),cog
307,Shure SE215,1.9,2.67,3,3.0,2.0,1.00,1.0,1.0,1.0,...,A relic of the past that should be avoided by ...,D-,D+,E+,D,1.9 (D-),2.67 (D+),1.0 (E+),2.0 (D),cog
308,Periodic Audio Ti,1.2,1.67,2,2.0,1.0,1.00,1.0,1.0,1.0,...,"Wait, this is the intended sound? My unit isn'...",E+,D-,E+,E+,1.2 (E+),1.67 (D-),1.0 (E+),1.0 (E+),cog


In [178]:
gizdf=pd.read_csv("https://docs.google.com/spreadsheets/d/1HFCuUzWdheP5qbxIJhyezJ53hvwM0wMrptVxKo49AFI/export?format=csv")
gizdf.columns = gizdf.iloc[0]
gizdf = gizdf.iloc[1:]
gizdf = gizdf[gizdf['NAME'].notna()]
gizdf = gizdf.reset_index(drop=True)
# Temporarily only look at the first 6 columns because I don't want to format the rest of the info.
gizdf = gizdf.drop(gizdf.columns[6:], axis=1)
gizdf = gizdf.rename(columns={'NAME':'Model', 'RANKING':'Normalized Grade', 'PRICE (USD)':'Price', "Doesn't effect rank (max 10)":'Preference Float'})
gizdf.columns

Index(['Normalized Grade', 'Model', 'PROS', 'CONS', 'Price',
       'Preference Float'],
      dtype='object', name=0)

In [179]:
gizdf['Normalized Float'] = gizdf['Normalized Grade'].replace({'S+':9.0, 'S':8.5, 'S-':8.0, 'A+':7.5, 'A':7.0, 'A-':6.5, 'B+':6.0, 'B':5.5, 'B-':5.0, 'C+':4.5, 'C':4.0, 'C-':3.5, 'D+':3.0, 'D':2.5, 'D-':2.0, 'E+':1.5, 'E':1.0, 'E-':0.5, 'F':0})
gizdf['Normalized Float and Grade'] = gizdf['Normalized Float'].astype(str) + ' (' + gizdf['Normalized Grade'].astype(str) + ')'
emoji_pattern = r'([\U00002600-\U000027BF]|\U0001f300-\U0001f64F|\U0001f680-\U0001f6FF|\U0001F700-\U0001F77F|\U0001F780-\U0001F7FF|\U0001F800-\U0001F8FF|\U0001F900-\U0001F9FF|\U0001FA00-\U0001FA6F|\U0001FA70-\U0001FAFF|\U00002B50)'
gizdf['Price'] = gizdf['Price'].str.replace(emoji_pattern, '', regex=True)
gizdf['Preference Float']=gizdf['Preference Float'].str.replace(emoji_pattern, '', regex=True)
gizdf['Preference Float'] = gizdf['Preference Float'].astype(float)
gizdf['Preference Grade'] = gizdf['Preference Float'].apply(lambda x: 'S+' if x>=9.0 else 'S' if x >= 8.7 else 'S-' if x >= 7.9 else 'A+' if x >= 7 else 'A' if x >= 6.5 else 'A-' if x >= 6 else 'B+' if x >= 5.5 else 'B' if x >= 5 else 'B-' if x >= 4.5 else 'C+' if x >= 4 else 'C' if x >= 3.5 else 'C-' if x >= 3 else 'D+' if x >= 2.5 else 'D' if x >= 2 else 'D-' if x >= 1.5 else 'E+' if x >= 1 else 'E' if x >= 0.5 else 'E-' if x >= 0.2 else 'F+' if x >= 0.1 else 'F')
def combine_pros_and_cons(row):
    if pd.isna(row['PROS']) and pd.isna(row['CONS']):
        return 'No comments'
    # If there are only pros or cons, return what is available.
    elif pd.isna(row['PROS']):
        return row['CONS']
    elif pd.isna(row['CONS']):
        return row['PROS']
    # If there are both pros and cons return both.
    else:
        return 'PROS: ' + row['PROS'] + ' CONS: ' + row['CONS']
gizdf['Comments'] = gizdf.apply(combine_pros_and_cons, axis=1)
# remove the first row
gizdf=gizdf.iloc[1:]
gizdf=gizdf.reset_index(drop=True)
gizdf=gizdf.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
gizdf.head()
# remove any rows where any column contains string "Re-Rank"
gizdf = gizdf[~gizdf['Normalized Grade'].str.contains('Re-Rank')]
gizdf = gizdf[~gizdf['Normalized Grade'].str.contains('Total IEMs Ranked')]
# Add list column for gizdf
gizdf['List'] = 'giz'

In [180]:
# using rapidfuzz, find the closest match for each model in the gizdf dataframe, but only if the match is at least 90% similar.
# If the match is at least 90% similar, then add the model name from the iefdf dataframe to a dataframe.
# If no match is found, return the original model name.

def modelmatch(row):
    model = row['Model']
    match = rapidfuzz.process.extractOne(model, iefdf['Model'], score_cutoff=98)
    if match is not None:
        return match[0]
    else:
        return model

gizdf['Model'] = gizdf.apply(iefmatch, axis=1)
antdf['Model'] = antdf.apply(iefmatch, axis=1)
cogdf['Model'] = cogdf.apply(iefmatch, axis=1)

In [196]:
# congregate all the dataframes into one dataframe
frames = pd.concat([iefdf, antdf, cogdf, gizdf],axis=0)

In [197]:
name_variations={
    "Moondrop B2: Dusk":"Moondrop Blessing 2: Dusk",
    "Moondrop Dusk":"Moondrop Blessing 2: Dusk",
    "Moondrop Blessing 2 Dusk":"Moondrop Blessing 2: Dusk",
    "Elysian Annihilator":"Elysian Annihilator 2021",
    "Elysian Annihilator (2021)":"Elysian Annihilator 2021",
    "Campfire Andromeda":"Campfire Andromeda 2019",
    "QDC Anole VX":"qdc 8SL/Gemini/Anole VX",
    "Shuoer S12":"LETSHUOER S12"
    }
frames['Model'] = frames['Model'].replace(name_variations)

frames['Model'] = frames['Model'].apply(lambda x: "Hidition Viento-B" if x and ("Viento" in x) and ("B" in x) else x)

In [198]:
def fuzz_match(row):
    model = row['Model']
    match = rapidfuzz.process.extractOne(model, frames['Model'], score_cutoff=90)
    if match is not None:
        return match[0]
    else:
        return model
# Apply the fuzz_match function to the Model column
frames['Model'] = frames.apply(fuzz_match, axis=1)
frames=frames.reset_index(drop=True)

,Normalized Grade,Model,Price,Signature,Comments,Tone Grade,Tech Grade,Setup,Status,Normalized Float,...,Preference Float,Preference Float and Grade,Bass,Midrange,Treble,Detail,Imaging,Dynamics,PROS,CONS
0,S-,Elysian Annihilator 2021,3700,U-shaped,,S-,S,2EST 4BA 1DD,Zeppelin & Co demo unit,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S-,Hidition Viento-B,950,Variable,Best-in-class tuning and tonal balance with ne...,S+,A+,4BA,Zeppelin & Co demo unit,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,S-,ThieAudio Monarch Mk2,1000,Neutral with bass boost,,S+,A+,2EST 6BA 1DD,Available for demo at The Hangout,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A+,Hidition NT6,1050,Neutral,"If Etymotic made a multi-BA IEM, this would be...",S-,A+,6BA,Null Audio demo unit,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A+,ThieAudio V16 Divinity,1500,Neutral with bass boost,,S-,A+,16BA,Available for demo at The Hangout,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,,Muse Hifi Metaverse ME1,$200,NaN,"Bloat, unnatural timbre, painful treble",NaN,NaN,NaN,NaN,,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bloat, unnatural timbre, painful treble",nan
144,,FiiO FH5,$230,NaN,"Very blunt, bass leaning IEM with also low ove...",NaN,NaN,NaN,NaN,,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Very blunt, bass leaning IEM with also low ove...",nan
166,A-,Aful Performer 8,nan,NaN,No comments,NaN,NaN,NaN,NaN,6.5,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan
167,B-,SeeAudio Yume Ultra,$220,NaN,PROS: Clean and spacious CONS: A bit intense i...,NaN,NaN,NaN,NaN,5.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clean and spacious,A bit intense in vocal presence
